In [1]:
import itk
import itkwidgets
from itkwidgets import view
import tifffile
import numpy as np

In [3]:
from EmbedSeg.utils.create_dicts import create_test_configs_dict
from EmbedSeg.test import begin_evaluating
from glob import glob
import tifffile
import matplotlib.pyplot as plt
from EmbedSeg.utils.visualize import visualize
import os
from matplotlib.colors import ListedColormap
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Specify the path to the evaluation images

In [4]:
data_dir = '../../../data'
project_name = 'Mouse-Organoid-Cells-CBG'
print("Evaluation images shall be read from: {}".format(os.path.join(data_dir, project_name)))

Evaluation images shall be read from: ../../../data/Mouse-Organoid-Cells-CBG


### Specify evaluation parameters 

Some hints:
* `tta`: Setting this to True (default) would enable **test-time augmentation**
* `ap_val`: This parameter ("average precision value") comes into action if ground truth segmentations exist for evaluation images, and allows to compare how good our predictions are versus the available ground truth segmentations.
* `seed_thresh`: This parameter ("seediness threshold") allows considering only those pixels as potential instance-centres which have a seediness score greater than `seed_thresh`
* `min_object_size`: This parameter ("minimum object size") allows considering only those predicted instances which have more interior pixels than `min_object_size` 
* `checkpoint_path`: This parameter provides the path to the trained model weights which you would like to use for evaluation. One could test the pretrained model (available at `'../../../pretrained_models/Mouse-Organoid-Cells-CBG/best_iou_model.pth'`) to get a quick glimpse on the results.
* `save_dir`: This parameter specifies the path to the prediction instances. Equal to `static` by default.
* `save_images`: If True, this saves predictions at `./static/predictions/` 
* `save_results`: If True, this saves results at `./static/results/`
* `normalization_factor`: Set this to either 255 (for 8-bit raw-images) or 65535 (for 16-bit instance-images). This parameter should be **set the same value** as `normalization_factor` used while creating `train_dataset_dict` and `val_dataset_dict` during training.
* `one_hot`: In case, ground truth segmentations exist for evaluation images, then set this parameter to `True` if the GT segmentations are present in a one-hot encoded style (i.e. each object is encoded as 1 in its own individual slice and 0 elsewhere) or `False`, otherwise

In the cell after this one, a `test_configs` dictionary is generated from the parameters specified here!
<a id='checkpoint'></a>

In [5]:
tta = True
ap_val = 0.5
seed_thresh = 0.90
min_object_size = 36
#checkpoint_path = os.path.join('experiment', project_name+'-'+'demo', 'best_iou_model.pth')
checkpoint_path = os.path.join('../../../pretrained_models', project_name, 'best_iou_model.pth')
save_dir = './static'
save_images = True
save_results = True
normalization_factor = 65535
one_hot = False
n_z = 72
n_y = 408
n_x = 408
voxel_size = [0.1733, 0.1733, 1.0] #XYZ
name = '3d'
mask_start = [51, 0, 231]
mask_end = [72, 152, 408]
mask_intensity = 129/normalization_factor

In [6]:
if os.path.exists(checkpoint_path):
    print("Trained model weights found at : {}".format(checkpoint_path))
else:
    print("Trained model weights were not found at the specified location!")

Trained model weights found at : ../../../pretrained_models/Mouse-Organoid-Cells-CBG/best_iou_model.pth


### Create `test_configs` dictionary from the above-specified parameters

In [7]:
test_configs = create_test_configs_dict(data_dir = os.path.join(data_dir, project_name),
                                        checkpoint_path = checkpoint_path,
                                        tta = tta, 
                                        ap_val = ap_val,
                                        seed_thresh = seed_thresh, 
                                        min_object_size = min_object_size, 
                                        save_images = save_images,
                                        save_results = save_results,
                                        save_dir = save_dir,
                                        normalization_factor = normalization_factor,
                                        one_hot = one_hot,
                                        n_z = n_z,
                                        n_y = n_y,
                                        n_x = n_x,
                                        anisotropy_factor = voxel_size[2]/voxel_size[0],
                                        name = name,
                                        )

`test_configs` dictionary successfully created with: 
 -- evaluation images accessed from ../../../data/Mouse-Organoid-Cells-CBG, 
 -- trained weights accessed from ../../../pretrained_models/Mouse-Organoid-Cells-CBG/best_iou_model.pth, 
 -- seediness threshold set at 0.9, 
 -- output directory chosen as ./static


### Begin Evaluating

Setting `verbose` to True shows you Average Precision at IOU threshold specified by `ap_val` above for each individual image. The higher this score is, the better the network has learnt to perform instance segmentation on these unseen images.

In [ ]:
begin_evaluating(test_configs, verbose = False, mask = True, mask_region = [mask_start, mask_end], mask_intensity = mask_intensity)

<div class="alert alert-block alert-warning"> 
  Common causes for a low score/error is: <br>
    1. Accessing the model weights at the wrong location. Simply editing the <b> checkpoint_path</b> would fix the issue. <br>
    2. At times, you would notice an improved performance by lowering <b><a href="#checkpoint"> seed_thresh</a></b> from 0.90 (default) to say 0.80. <br>
    3. CUDA error: out of memory - ensure that you shutdown <i>02-train.ipynb</i> notebook before running this notebook.
</div>

### Visualize some predictions

Here, we use the `itkwidgets` to first display any one of the evaluation images and then display the corresponding prediction by the model. Please feel free to change the `index` to look at other predictions.

In [15]:
if(save_images):
    prediction_file_names = sorted(glob(os.path.join(save_dir,'predictions','*.tif')))
    ground_truth_file_names = sorted(glob(os.path.join(save_dir,'ground-truth','*.tif')))
    image_file_names = sorted(glob(os.path.join(data_dir, project_name, 'test', 'images','*.tif')))

In [18]:
index = 8
print("Image filename is {} and index is {}".format(os.path.basename(image_file_names[index]), index))
prediction = tifffile.imread(prediction_file_names[index])
ground_truth = tifffile.imread(ground_truth_file_names[index])
image = tifffile.imread(image_file_names[index])
image_itk =itk.GetImageFromArray(image)
image_itk.SetSpacing(voxel_size)
view(image_itk, cmap=itkwidgets.cm.BrBG, annotations=False, vmax=800, ui_collapsed=True)


Image filename is X164.tif and index is 8


Viewer(annotations=False, cmap=['BrBG'], geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<…

In [19]:
prediction_itk =itk.GetImageFromArray(prediction)
prediction_itk.SetSpacing(voxel_size)
view(prediction_itk, cmap=itkwidgets.cm.BrBG, annotations=False, vmax=800, ui_collapsed=True)

Viewer(annotations=False, cmap=['BrBG'], geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<…